In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dvu
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import neuro.features.feature_utils
import joblib
import neuro.data.story_names
import neuro.data.response_utils
from tqdm import tqdm
import neuro.features.feature_spaces
from himalaya.ridge import RidgeCV

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### set up basic regression problem

In [56]:
subject = 'UTS03'
n_train_stories = 30
n_voxels = 100
n_feats = 1000

In [ ]:
story_names_train = neuro.data.story_names.get_story_names(
    subject=subject, train_or_test='train')[:n_train_stories]
story_names_test = neuro.data.story_names.get_story_names(
    subject=subject, train_or_test='test')
resps_train = neuro.data.response_utils.load_response(
    story_names_train, subject=subject)
resps_test = neuro.data.response_utils.load_response(
    story_names_test, subject=subject)

In [ ]:
class A:
    feature_space = 'eng1000'


args = A()
feats_train = neuro.features.feature_utils.get_features_full(
    args=args,
    feature_space=args.feature_space,
    qa_embedding_model=None,
    story_names=story_names_train,
    use_added_delays=False,
)
feats_test = neuro.features.feature_utils.get_features_full(
    args=args,
    feature_space=args.feature_space,
    qa_embedding_model=None,
    story_names=story_names_test,
    use_added_delays=False,
)

print('feat shapes', feats_train.shape, feats_test.shape)
print('resp shapes', resps_train.shape, resps_test.shape)

feat shapes (864, 985) (639, 985)
resp shapes (864, 95556) (639, 95556)


In [ ]:
# subsample output voxels and feature inputs
rng = np.random.default_rng(0)
random_voxels = rng.choice(
    np.arange(resps_train.shape[1]), size=100, replace=False
)
resps_train_subsampled_voxels = resps_train[:, random_voxels]
resps_test_subsampled_voxels = resps_test[:, random_voxels]

feats_train_subsampled = feats_train[:, :n_feats]
feats_test_subsampled = feats_test[:, :n_feats]

In [ ]:
model = RidgeCV(alphas=[1, 10, 100])
model.fit(feats_train_subsampled, resps_train_subsampled_voxels)
preds_train = model.predict(feats_train_subsampled)
preds_test = model.predict(feats_test_subsampled)


def get_mean_corr(preds, resps):
    return np.mean([np.corrcoef(preds[:, i], resps[:, i])[0, 1] for i in range(preds.shape[1])])


print('train', get_mean_corr(preds_train, resps_train_subsampled_voxels))
print('test', get_mean_corr(preds_test, resps_test_subsampled_voxels))

/home/chansingh/.env/lib/python3.11/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 864 < 985). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(


train 0.7258420510695542
test 0.009865030391749975
